### Moving Average Crossover Strategy

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# mac.py

from __future__ import print_function

import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm

from scripts.strategy import Strategy
from scripts.event import SignalEvent
from scripts.backtest import Backtest
from scripts.data import HistoricCSVDataHandler
from scripts.execution import SimulatedExecutionHandler
from scripts.portfolio import Portfolio


class MovingAverageCrossStrategy(Strategy):
        """
        Carries out a basic Moving Average Crossover strategy with a
        short/long simple weighted moving average. Default short/long
        windows are 100/400 periods respectively.
        """
        
        def __init__(self, bars, events, short_window=100, long_window=400):
            """
            Initialises the Moving Average Cross Strategy.
            Parameters:
            bars - The DataHandler object that provides bar information
            events - The Event Queue object.
            short_window - The short moving average lookback.
            long_window - The long moving average lookback.
            """
        
            self.bars = bars
            self.symbol_list = self.bars.symbol_list
            self.events = events
            self.short_window = short_window
            self.long_window = long_window
            
            # Set to True if a symbol is in the market
            self.bought = self._calculate_initial_bought()
            
            
        def _calculate_initial_bought(self):
            """
            Adds keys to the bought dictionary for all symbols
            and sets them to ’OUT’.
            """
            
            bought = {}
            
            for s in self.symbol_list:
                bought[s] = 'OUT'
            
            return bought
        
        
        def calculate_signals(self, event):
            """
            Generates a new set of signals based on the MAC
            SMA with the short window crossing the long window
            meaning a long entry and vice versa for a short entry.
            Parameters
            event - A MarketEvent object.
            """
            
            if event.type == 'MARKET':
                for s in self.symbol_list:
                    bars = self.bars.get_latest_bars_values(s, "adj_close", N=self.long_window)
                    bar_date = self.bars.get_latest_bar_datetime(s)
                
                    if bars is not None and bars != []:
                        short_sma = np.mean(bars[-self.short_window:])
                        long_sma = np.mean(bars[-self.long_window:])
                        
                        symbol = s
                        dt = datetime.datetime.utcnow()
                        sig_dir = ""
                    
                        if short_sma > long_sma and self.bought[s] == "OUT":
                            print("LONG: %s" % bar_date)
                            sig_dir = 'LONG'
                            signal = SignalEvent(1, symbol, dt, sig_dir, 1.0)
                            self.events.put(signal)
                            self.bought[s] = 'LONG'
                        
                        elif short_sma < long_sma and self.bought[s] == "LONG":
                            print("SHORT: %s" % bar_date)
                            sig_dir = 'EXIT'
                            signal = SignalEvent(1, symbol, dt, sig_dir, 1.0)
                            self.events.put(signal)
                            self.bought[s] = 'OUT'


if __name__ == "__main__":        
    csv_dir = 'stock_data/' # CHANGE THIS!
    symbol_list = ['AAPL']
    initial_capital = 100000.0
    heartbeat = 0.0
    start_date = datetime.datetime(1990, 1, 1, 0, 0, 0)
    backtest = Backtest(csv_dir, 
                        symbol_list, 
                        initial_capital, 
                        heartbeat,
                        start_date,
                        HistoricCSVDataHandler,
                        SimulatedExecutionHandler,
                        Portfolio,
                        MovingAverageCrossStrategy)
    
    backtest.simulate_trading()

### S&P500 Forecasting Trade

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# snp_forecast.py

from __future__ import print_function

import datetime
import pandas as pd

from sklearn.qda import QDA

from scripts.strategy import Strategy
from scripts.event import SignalEvent
from scripts.backtest import Backtest
from scripts.data import HistoricCSVDataHandler
from scripts.execution import SimulatedExecutionHandler
from scripts.portfolio import Portfolio

from create_lagged_series import create_lagged_series


class SPYDailyForecastStrategy(Strategy):
    """S&P500 forecast strategy. It uses a Quadratic Discriminant
    Analyser to predict the returns for a subsequent time
    period and then generated long/exit signals based on the
    prediction.
    """

    def __init__(self, bars, events):
        self.bars = bars
        self.symbol_list = self.bars.symbol_list
        self.events = events
        self.datetime_now = datetime.datetime.utcnow()
        self.model_start_date = datetime.datetime(2001,1,10)
        self.model_end_date = datetime.datetime(2005,12,31)
        self.model_start_test_date = datetime.datetime(2005,1,1)
        self.long_market = False
        self.short_market = False
        self.bar_index = 0
        self.model = self.create_symbol_forecast_model()

    
    
    def create_symbol_forecast_model(self):
        
        # Create a lagged series of the S&P500 US stock market index
        snpret = create_lagged_series(self.symbol_list[0], 
                                      self.model_start_date,
                                      self.model_end_date, lags=5)
        
        # Use the prior two days of returns as predictor
        # values, with direction as the response
        X = snpret[["Lag1","Lag2"]]
        y = snpret["Direction"]
        
        # Create training and test sets
        start_test = self.model_start_test_date
        X_train = X[X.index < start_test]
        X_test = X[X.index >= start_test]
        y_train = y[y.index < start_test]
        y_test = y[y.index >= start_test]
        model = QDA()
        model.fit(X_train, y_train)
        
        return model

    def calculate_signals(self, event):
        """
        Calculate the SignalEvents based on market data.
        """
        
        sym = self.symbol_list[0]
        dt = self.datetime_now
        
        if event.type == 'MARKET':
            self.bar_index += 1
            if self.bar_index > 5:
                lags = self.bars.get_latest_bars_values(
                self.symbol_list[0], "returns", N=3)
                pred_series = pd.Series({'Lag1': lags[1]*100.0,'Lag2': lags[2]*100.0})
                pred = self.model.predict(pred_series)
            
                if pred > 0 and not self.long_market:
                    self.long_market = True
                    signal = SignalEvent(1, sym, dt, 'LONG', 1.0)
                    self.events.put(signal)
                
                if pred < 0 and self.long_market:
                    self.long_market = False
                    signal = SignalEvent(1, sym, dt, 'EXIT', 1.0)
                    self.events.put(signal)


if __name__ == "__main__":        
    csv_dir = 'stock_data/' # CHANGE THIS!
    symbol_list = ['SPY']
    initial_capital = 100000.0
    heartbeat = 0.0
    start_date = datetime.datetime(2010,1,3)
    backtest = Backtest(csv_dir, 
                        symbol_list, 
                        initial_capital, 
                        heartbeat,
                        start_date, 
                        HistoricCSVDataHandler,
                        SimulatedExecutionHandler,
                        Portfolio,
                        SPYDailyForecastStrategy)
    
    backtest.simulate_trading()

### Mean-Reverting Equity Pairs Trade

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# intraday_mr.py

from __future__ import print_function

import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm

from scripts.strategy import Strategy
from scripts.event import SignalEvent
from scripts.backtest import Backtest

from hft_data import HistoricCSVDataHandlerHFT
from hft_portfolio import PortfolioHFT
from scripts.execution import SimulatedExecutionHandler


class IntradayOLSMRStrategy(Strategy):
    """
    Uses ordinary least squares (OLS) to perform a rolling linear
    regression to determine the hedge ratio between a pair of equities.
    The z-score of the residuals time series is then calculated in a
    rolling fashion and if it exceeds an interval of thresholds
    (defaulting to [0.5, 3.0]) then a long/short signal pair are generated
    (for the high threshold) or an exit signal pair are generated (for the
    low threshold).
    """
    
    def __init__(self, bars, events, 
                 ols_window=100,
                 zscore_low=0.5, 
                 zscore_high=3.0):
        """
        Initialises the stat arb strategy.
        Parameters:
        bars - The DataHandler object that provides bar information
        events - The Event Queue object.
        """
    
        self.bars = bars
        self.symbol_list = self.bars.symbol_list
        self.events = events
        self.ols_window = ols_window
        self.zscore_low = zscore_low
        self.zscore_high = zscore_high
        self.pair = ('AREX', 'WLL')
        self.datetime = datetime.datetime.utcnow()
        self.long_market = False
        self.short_market = False
        
    
    def calculate_xy_signals(self, zscore_last):
        """
        Calculates the actual x, y signal pairings
        to be sent to the signal generator.

        Parameters
        zscore_last - The current zscore to test against
        """

        y_signal = None
        x_signal = None
        p0 = self.pair[0]
        p1 = self.pair[1]
        dt = self.datetime
        hr = abs(self.hedge_ratio)

        # If we’re long the market and below the
        # negative of the high zscore threshold
        if zscore_last <= -self.zscore_high and not self.long_market:
            self.long_market = True
            y_signal = SignalEvent(1, p0, dt, 'LONG', 1.0)
            x_signal = SignalEvent(1, p1, dt, 'SHORT', hr)

        # If we’re long the market and between the
        # absolute value of the low zscore threshold
        if abs(zscore_last) <= self.zscore_low and self.long_market:
            self.long_market = False
            y_signal = SignalEvent(1, p0, dt, 'EXIT', 1.0)
            x_signal = SignalEvent(1, p1, dt, 'EXIT', 1.0)

        # If we’re short the market and above
        # the high zscore threshold
        if zscore_last >= self.zscore_high and not self.short_market:
            self.short_market = True
            y_signal = SignalEvent(1, p0, dt, 'SHORT', 1.0)
            x_signal = SignalEvent(1, p1, dt, 'LONG', hr)

        # If we’re short the market and between the
        # absolute value of the low zscore threshold
        if abs(zscore_last) <= self.zscore_low and self.short_market:
            self.short_market = False
            y_signal = SignalEvent(1, p0, dt, 'EXIT', 1.0)
            x_signal = SignalEvent(1, p1, dt, 'EXIT', 1.0)

        return y_signal, x_signal

    def calculate_signals_for_pairs(self):
        """
        Generates a new set of signals based on the mean reversion
        strategy.

        Calculates the hedge ratio between the pair of tickers.
        We use OLS for this, althought we should ideall use CADF.
        """
        
        # Obtain the latest window of values for each
        # component of the pair of tickers
        y = self.bars.get_latest_bars_values(self.pair[0], "close", N=self.ols_window)
        x = self.bars.get_latest_bars_values(self.pair[1], "close", N=self.ols_window)
        
        if y is not None and x is not None:
            
            # Check that all window periods are available
            if len(y) >= self.ols_window and len(x) >= self.ols_window:
                
                # Calculate the current hedge ratio using OLS
                self.hedge_ratio = sm.OLS(y, x).fit().params[0]
                
                # Calculate the current z-score of the residuals
                spread = y - self.hedge_ratio * x
                zscore_last = ((spread - spread.mean())/spread.std())[-1]
                
                # Calculate signals and add to events queue
                y_signal, x_signal = self.calculate_xy_signals(zscore_last)
                if y_signal is not None and x_signal is not None:
                    self.events.put(y_signal)
                    self.events.put(x_signal)

    def calculate_signals(self, event):
        """
        Calculate the SignalEvents based on market data.
        """

        if event.type == 'MARKET':
            self.calculate_signals_for_pairs()


if __name__ == "__main__":        
    csv_dir = 'stock_data/' # CHANGE THIS!
    symbol_list = ['ARE', 'WLL']
    initial_capital = 100000.0
    heartbeat = 0.0
    start_date = datetime.datetime(2010, 11, 8, 10, 41, 0)
    backtest = Backtest(csv_dir, 
                        symbol_list, 
                        initial_capital, 
                        heartbeat,
                        start_date,
                        HistoricCSVDataHandlerHFT,
                        SimulatedExecutionHandler,
                        PortfolioHFT,
                        IntradayOLSMRStrategy)
    
    backtest.simulate_trading()



### End.